
# Weather Classification Analysis for Wildfire Avoidance

As an Environmental Engineer deeply invested in leveraging data science for environmental recovery, especially in high-risk areas like those prone to wildfires, this project resonates with my core mission. My analytical approach drives me to break down complex environmental challenges into manageable data problems. Here, I apply machine learning to predict dry weather conditions, a critical step in proactive wildfire prevention and resource management. This work is a step-by-step exploration, connecting technical skills with the vital goal of protecting our ecosystems and communities.

This notebook performs data preparation, including basic inspection, cleaning, exploratory data analysis (EDA), and feature engineering, for a weather classification dataset. The goal is to predict dryness trends for wildfire avoidance, with 'Dry' conditions defined based on parameters relevant to the Canadian Forest Fire Weather Index (FWI) system. It also includes statistical tests to understand the relationships between features and dryness.


In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats # For statistical tests
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
df = pd.read_csv("weather_classification_data.csv")



## 1. Basic Inspection

My first step in any data project is to get a foundational understanding of the dataset. This involves a quick look at the first few rows, checking the overall size, and examining the data types to ensure everything is as expected. It's like surveying the landscape before starting an environmental assessment.


In [3]:

print("---Checking Head Rows---")
df.head()


---Checking Head Rows---


,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location,Weather Type
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland,Rainy
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland,Cloudy
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain,Sunny
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal,Sunny
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain,Rainy


In [4]:
print("\n---Checking Rows and Columns---")
print(df.shape)
print("\n---Checking Data Types---")
print(df.dtypes)
print("\n---Checking Dataset Info---")
df.info()


---Checking Rows and Columns---
(13200, 11)

---Checking Data Types---
Temperature             float64
Humidity                  int64
Wind Speed              float64
Precipitation (%)       float64
Cloud Cover              object
Atmospheric Pressure    float64
UV Index                  int64
Season                   object
Visibility (km)         float64
Location                 object
Weather Type             object
dtype: object

---Checking Dataset Info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13200 entries, 0 to 13199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Temperature           13200 non-null  float64
 1   Humidity              13200 non-null  int64  
 2   Wind Speed            13200 non-null  float64
 3   Precipitation (%)     13200 non-null  float64
 4   Cloud Cover           13200 non-null  object 
 5   Atmospheric Pressure  13200 non-null  float64
 6   UV Index   


## 2. Data Cleaning

Just as environmental remediation starts with removing contaminants, data cleaning is about ensuring the integrity of our dataset. Here, I focus on identifying and addressing duplicate entries and missing values, which can skew our analysis and model performance.


In [5]:

print("---Checking for duplicates---")
print(f"{df.duplicated().sum()} duplicates found.")
df.drop_duplicates(inplace=True)
print(f"Shape after dropping duplicates: {df.shape}")

# Check for missing values
print("\nMissing values per column:")
print(df.isnull().sum())


---Checking for duplicates---
0 duplicates found.
Shape after dropping duplicates: (13200, 11)

Missing values per column:
Temperature             0
Humidity                0
Wind Speed              0
Precipitation (%)       0
Cloud Cover             0
Atmospheric Pressure    0
UV Index                0
Season                  0
Visibility (km)         0
Location                0
Weather Type            0
dtype: int64



## 3. Feature Engineering: Define Dryness Target Variable for Wildfire Avoidance

This is where I connect the raw weather data to a real-world environmental problem: wildfire risk. By defining 'Dry' conditions based on principles from the Canadian Forest Fire Weather Index (FWI) system, I'm creating a target variable that directly supports our goal of wildfire avoidance. It's about translating environmental indicators into actionable data points.



#### Based on FWI system inputs: temperature, relative humidity, wind speed, and 24-hour precipitation.

*   A simplified definition of 'Dry' for wildfire avoidance:
    -   Low Precipitation
    -   Low Humidity
    -   High Temperature
    -   Potentially High Wind Speed (though not directly used in this simple binary definition)

*   Define thresholds for 'Dry' conditions based on FWI inputs (these are illustrative and can be refined)
*   Let's consider 'Dry' if:
    -   Precipitation (%) < 10 (very low precipitation)
    -   Humidity < 50 (low humidity)
    -   Temperature > 25 (high temperature)



    __More details for FWI system on the PDF report__


In [6]:

def define_dryness_for_wildfire(row):
    if row["Precipitation (%)"] < 10 and row["Humidity"] < 50 and row["Temperature"] > 25:
        return "Dry"
    else:
        return "Not Dry"

df["Dryness_Label_Wildfire"] = df.apply(define_dryness_for_wildfire, axis=1)
print("\nValue counts for Dryness_Label_Wildfire:")
print(df["Dryness_Label_Wildfire"].value_counts())

# Target variable for classification (1 for Dry, 0 for Not Dry)
y = (df["Dryness_Label_Wildfire"] == "Dry").astype(int)

# Features (excluding original Weather Type and the new Dryness_Label_Wildfire)
x = df.drop(["Weather Type", "Dryness_Label_Wildfire"], axis=1)



Value counts for Dryness_Label_Wildfire:
Dryness_Label_Wildfire
Not Dry    12618
Dry          582
Name: count, dtype: int64


In [7]:
print(df["Dryness_Label_Wildfire"].unique())

print(y.unique())

['Not Dry' 'Dry']
[0 1]



## 4. Exploratory Data Analysis (EDA)

This section is my deep dive into the data, much like a field investigation. I use visualizations and statistical tests to uncover patterns and relationships, especially those that shed light on wildfire risk factors. Understanding these connections is crucial for building a robust predictive model and ensuring our insights are grounded in evidence.


In [8]:
x.head()

,Temperature,Humidity,Wind Speed,Precipitation (%),Cloud Cover,Atmospheric Pressure,UV Index,Season,Visibility (km),Location
0,14.0,73,9.5,82.0,partly cloudy,1010.82,2,Winter,3.5,inland
1,39.0,96,8.5,71.0,partly cloudy,1011.43,7,Spring,10.0,inland
2,30.0,64,7.0,16.0,clear,1018.72,5,Spring,5.5,mountain
3,38.0,83,1.5,82.0,clear,1026.25,7,Spring,1.0,coastal
4,27.0,74,17.0,66.0,overcast,990.67,1,Winter,2.5,mountain


In [9]:

print("\n--- Exploratory Data Analysis (Visualizations and Statistical Tests) ---")

# Identify numerical and categorical features
numerical_features = x.select_dtypes(include=np.number).columns
categorical_features = x.select_dtypes(include="object").columns

# Outlier Handling (Capping using IQR method)
# This approach helps to mitigate the impact of extreme values without removing data points,
# which is often preferred in environmental data where outliers might represent real, albeit rare, events.
for col in numerical_features:
    Q1 = df[col].quantile(0.20)
    Q3 = df[col].quantile(0.80)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower_bound, lower_bound, df[col])
    df[col] = np.where(df[col] > upper_bound, upper_bound, df[col])
    print(f"Outliers in {col} handled by capping.")

# Histograms for numerical features (after outlier handling)
# for col in numerical_features:
#     plt.figure(figsize=(8, 5))
#     sns.histplot(df[col], kde=True)
#     plt.title(f"Distribution of {col} (After Outlier Capping)")
#     #plt.show()

# # Box plots for numerical features comparing with Dryness_Label_Wildfire feature
# for col in numerical_features:
#     plt.figure(figsize=(8, 5))
#     sns.boxplot(x=df["Dryness_Label_Wildfire"], y=df[col])
#     plt.title(f"{col} by Dryness Label for Wildfire (After Outlier Capping)")
#     #plt.show()

# # Plots for categorical features
# for col in categorical_features:
#     plt.figure(figsize=(10, 6))
#     sns.countplot(data=df, y=col, hue="Dryness_Label_Wildfire", order = df[col].value_counts().index)
#     plt.title(f"Count of {col} by Dryness Label for Wildfire")
#     plt.tight_layout()
#     #plt.show()

# # Correlation heatmap
# plt.figure(figsize=(12, 10))
# correlation_matrix = df[numerical_features].corr()
# sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
# plt.title("Correlation Heatmap of Numerical Features")
# #plt.show()

# --- Statistical Tests for Dryness --- 
print("\n--- Statistical Tests for Dryness ---")

dry_group = df[df["Dryness_Label_Wildfire"] == "Dry"]
not_dry_group = df[df["Dryness_Label_Wildfire"] == "Not Dry"]

# T-tests for key numerical features (Temperature, Humidity, Precipitation (%), Wind Speed)
# to see if their means are significantly different between Dry and Not Dry groups.
# Assuming data is not strictly normal, but t-test is robust for large samples.
# For non-normal data, Mann-Whitney U test could be used.
key_fwi_features = ["Temperature", "Humidity", "Precipitation (%)", "Wind Speed"]

for feature in key_fwi_features:
    stat, p = stats.ttest_ind(dry_group[feature], not_dry_group[feature], equal_var=False) # t-test
    print(f"\n{feature} - T-test (Dry vs. Not Dry):")
    print(f"  Statistic: {stat:.3f}")
    print(f"  P-value: {p:.3f}")
    if p < 0.05:
        print(f"  Conclusion: Significant difference in {feature} between Dry and Not Dry groups (p < 0.05)")
    else:
        print(f"  Conclusion: No significant difference in {feature} between Dry and Not Dry groups (p >= 0.05)")

# Chi-squared test for categorical features (Season, Location, Cloud Cover) vs. Dryness_Label_Wildfire
print("\n--- Chi-squared Tests for Categorical Features vs. Dryness ---")
for feature in categorical_features:
    contingency_table = pd.crosstab(df[feature], df["Dryness_Label_Wildfire"])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    print(f"\n{feature} - Chi-squared test (vs. Dryness_Label_Wildfire):")
    print(f"  Chi2 Statistic: {chi2:.3f}")
    print(f"  P-value: {p:.3f}")
    if p < 0.05:
        print(f"  Conclusion: Significant association between {feature} and Dryness_Label_Wildfire (p < 0.05)")
    else:
        print(f"  Conclusion: No significant association between {feature} and Dryness_Label_Wildfire (p >= 0.05)")



--- Exploratory Data Analysis (Visualizations and Statistical Tests) ---
Outliers in Temperature handled by capping.
Outliers in Humidity handled by capping.
Outliers in Wind Speed handled by capping.
Outliers in Precipitation (%) handled by capping.
Outliers in Atmospheric Pressure handled by capping.
Outliers in UV Index handled by capping.
Outliers in Visibility (km) handled by capping.

--- Statistical Tests for Dryness ---

Temperature - T-test (Dry vs. Not Dry):
  Statistic: 60.965
  P-value: 0.000
  Conclusion: Significant difference in Temperature between Dry and Not Dry groups (p < 0.05)

Humidity - T-test (Dry vs. Not Dry):
  Statistic: -87.417
  P-value: 0.000
  Conclusion: Significant difference in Humidity between Dry and Not Dry groups (p < 0.05)

Precipitation (%) - T-test (Dry vs. Not Dry):
  Statistic: -172.391
  P-value: 0.000
  Conclusion: Significant difference in Precipitation (%) between Dry and Not Dry groups (p < 0.05)

Wind Speed - T-test (Dry vs. Not Dry):
  

---


## 5. Preprocessing

This final data preparation step transforms our raw and engineered features into a format suitable for machine learning models. It involves two key processes: scaling numerical data and encoding categorical data. Think of it as preparing different types of environmental samples for laboratory analysis—each needs specific handling to yield comparable and accurate results.


In [10]:

print("\n--- Preprocessing ---")

# Scaling Numerical Features: StandardScaler
# Numerical features like Temperature or Humidity have different ranges. 
numerical_transformer = StandardScaler()

# Encoding Categorical Features: OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore", drop="first")
        
# Combining Transformations with ColumnTransformer
# Apply different transformations for different columns of the dataset simultaneously.
# It ensures that numerical features are scaled and categorical features are encoded.
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ],
    remainder="passthrough" # Keep any other columns that weren't specified
)

# Apply preprocessing to features X
x_processed = preprocessor.fit_transform(x)

# Convert the processed array back to a DataFrame for easier viewing and further steps
# This also helps in understanding the new feature names created by OneHotEncoder.
feature_names_out = preprocessor.get_feature_names_out()
x_processed_df = pd.DataFrame(x_processed, columns=feature_names_out)

print("\nShape of processed X:")
print(x_processed_df.shape)



--- Preprocessing ---

Shape of processed X:
(13200, 15)


##### Encoded and Scaled Dataset 

In [11]:
x_processed_df.head()

,num__Temperature,num__Humidity,num__Wind Speed,num__Precipitation (%),num__Atmospheric Pressure,num__UV Index,num__Visibility (km),cat__Cloud Cover_cloudy,cat__Cloud Cover_overcast,cat__Cloud Cover_partly cloudy,cat__Season_Spring,cat__Season_Summer,cat__Season_Winter,cat__Location_inland,cat__Location_mountain
0,-0.294931,0.212404,-0.048086,0.887629,0.134203,-0.520104,-0.582231,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.143035,1.351385,-0.192836,0.543291,0.150602,0.776424,1.345768,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,0.625367,-0.233285,-0.409962,-1.178401,0.346579,0.257813,0.010999,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.085516,0.707613,-1.206089,0.887629,0.549008,0.776424,-1.323769,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.452811,0.261924,1.037543,0.386773,-0.407490,-0.779410,-0.878846,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [12]:
x.shape

(13200, 10)

In [13]:
x = x_processed_df
# Save the processed features and target variable for future use

---

### Machine learning 

* ML model selection
    - At this point of the project, I'll start the process to perform the __Multiple Linear Regression__ to practice and consolidate the knowledge acquired during the classes.

Correlation heatmap

In [14]:
# plt.figure(figsize=(12, 10))
# sns.heatmap(x_processed_df.corr(), annot=True, fmt=".3f")
# plt.title("Correlation Heatmap of Processed Features")
# plt.show()
# #Check for multicollinearity using VIF
# #Create a dataframe which has column names and associated VIF value for each column.


In [15]:
#Check for multicollinearity using VIF
#Create a dataframe which has column names and associated VIF value for each column.
vif_df = pd.DataFrame({'Variable': x.columns, 
                       'VIF' : [vif(x.values, i) for i in range(x.shape[1])]
                      })

vif_df

,Variable,VIF
0,num__Temperature,1.440491
1,num__Humidity,1.973567
2,num__Wind Speed,1.345921
3,num__Precipitation (%),2.072371
4,num__Atmospheric Pressure,1.069968
5,num__UV Index,1.461451
6,num__Visibility (km),1.554803
7,cat__Cloud Cover_cloudy,1.202969
8,cat__Cloud Cover_overcast,3.571049
9,cat__Cloud Cover_partly cloudy,2.274937


Because any of the VIF variable values are greater than 5 __THERE IS NO MULTICOLINEARITY__ between the predictables on the dataset.  

#### TRAINING & TEST SPLIT 

In [16]:
X_40 = x.copy()
Y_40 = y.copy()

60/40

In [ ]:
#Split the data
x_40_train, x_40_test, Y_40_train, Y_40_test = train_test_split(x, y, test_size=0.4, random_state = 42)

In [18]:
x_40_train.shape

(7920, 15)

In [19]:
x_40_test.shape

(5280, 15)

In [20]:
#Multiple Linear Regression
lr_40 = LinearRegression() #Model Building
lr_40.fit(x_40_train, Y_40_train) #Model Training

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [21]:
Y_40_pred = lr_40.predict(x_40_test)
Y_40_pred

array([ 0.27610588,  0.02044391,  0.01424464, ..., -0.07775306,
        0.23643836, -0.03654082], shape=(5280,))

In [22]:
#Find the line slope
lr_40.coef_

array([ 0.01414223, -0.05368195,  0.00566977, -0.02550063, -0.00102719,
        0.0125581 , -0.01638866, -0.14720938, -0.0855019 , -0.08301649,
       -0.00349661, -0.00389906,  0.02113246,  0.01687418,  0.01223907])

In [23]:
#FInd the line intercept
lr_40.intercept_

np.float64(0.09857604720679215)

In [24]:
#Predict data
Y_40_pred = lr_40.predict(x_40_test)
Y_40_pred

array([ 0.27610588,  0.02044391,  0.01424464, ..., -0.07775306,
        0.23643836, -0.03654082], shape=(5280,))

In [25]:
Y_40_test.head()

1092     0
10991    0
6597     0
4164     0
445      0
Name: Dryness_Label_Wildfire, dtype: int64

In [26]:
#Dataframe for predicted and actual values
results_diff = pd.DataFrame({'Predicted Value': Y_40_pred, 'Actual Value': Y_40_test})
results_diff

,Predicted Value,Actual Value
1092,0.276106,0
10991,0.020444,0
6597,0.014245,0
4164,-0.079396,0
445,-0.006900,0
...,...,...
6316,0.092646,0
3013,0.024383,0
9880,-0.077753,0
3901,0.236438,0


In [27]:
results_diff['Actual Value'].unique()

array([0, 1])

In [28]:
# #Plot the histogram for the error/cost function
# fig = plt.figure()
# sns.histplot(Y_40_test - Y_40_pred)
# plt.xlabel('Y_test - Y_test_pred', fontsize = 15)
# plt.show()

In [29]:
#Compute the metrics
mae = mean_absolute_error(Y_40_test, Y_40_pred)
mse = mean_squared_error(Y_40_test, Y_40_pred)
r2 = r2_score(Y_40_test, Y_40_pred)

print('Mean Absolute Error: ', mae)
print('Mean Squared Error: ', mse)
print('R2 coefficient: ', r2)

Mean Absolute Error:  0.09633865022377901
Mean Squared Error:  0.035328118586070455
R2 coefficient:  0.18898860984741694


---

90/10

In [30]:
X_10 = x.copy()
Y_10 = y.copy()

In [ ]:
#Split the data
x_10_train, x_10_test, Y_10_train, Y_10_test = train_test_split(x, y, test_size=0.1, random_state = 42)

In [32]:
#Multiple Linear Regression
lr_10 = LinearRegression() #Model Building
lr_10.fit(x_10_train, Y_10_train) #Model Training

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [33]:
#Find the line slope
lr_10.coef_

array([ 0.01612639, -0.05419842,  0.00580897, -0.02620518, -0.00025183,
        0.01311301, -0.01721122, -0.1402461 , -0.0776065 , -0.07495092,
        0.00034552, -0.00023614,  0.02295847,  0.01529135,  0.0116529 ])

In [34]:
#FInd the line intercept
lr_10.intercept_

np.float64(0.09044501895904714)

In [35]:
#Predict data
Y_10_pred = lr_10.predict(x_10_test)
Y_10_pred

array([ 0.27121989,  0.02511164,  0.01050143, ...,  0.21290151,
       -0.05726193,  0.01970507], shape=(1320,))

In [36]:
#Dataframe for predicted and actual values
results_diff = pd.DataFrame({'Predicted Value': Y_10_pred, 'Actual Value': Y_10_test})
results_diff

,Predicted Value,Actual Value
1092,0.271220,0
10991,0.025112,0
6597,0.010501,0
4164,-0.079974,0
445,-0.008706,0
...,...,...
11395,0.155204,0
8142,-0.063436,0
11083,0.212902,0
281,-0.057262,0


In [37]:
# #Plot the histogram for the error/cost function
# fig = plt.figure()
# sns.histplot(Y_10_test - Y_10_pred)
# plt.xlabel('Y_test - Y_test_pred', fontsize = 15)
# plt.show()

In [38]:
#Compute the metrics
mae = mean_absolute_error(Y_10_test, Y_10_pred)
mse = mean_squared_error(Y_10_test, Y_10_pred)
r2 = r2_score(Y_10_test, Y_10_pred)

print('Mean Absolute Error: ', mae)
print('Mean Squared Error: ', mse)
print('R2 coefficient: ', r2)

Mean Absolute Error:  0.09809457264641738
Mean Squared Error:  0.0357862174127649
R2 coefficient:  0.20054996128004543


SIMPLE LINEAR REGRESSION: Making the simpler model possible to check if there is any problem on the Multiple Linear regression model performed above.

To do this test is necessary to select the most correlated features between the dataset, then it will be the most important predictable variable.

In [39]:
x_processed_df.head()

,num__Temperature,num__Humidity,num__Wind Speed,num__Precipitation (%),num__Atmospheric Pressure,num__UV Index,num__Visibility (km),cat__Cloud Cover_cloudy,cat__Cloud Cover_overcast,cat__Cloud Cover_partly cloudy,cat__Season_Spring,cat__Season_Summer,cat__Season_Winter,cat__Location_inland,cat__Location_mountain
0,-0.294931,0.212404,-0.048086,0.887629,0.134203,-0.520104,-0.582231,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,1.143035,1.351385,-0.192836,0.543291,0.150602,0.776424,1.345768,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,0.625367,-0.233285,-0.409962,-1.178401,0.346579,0.257813,0.010999,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,1.085516,0.707613,-1.206089,0.887629,0.549008,0.776424,-1.323769,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.452811,0.261924,1.037543,0.386773,-0.407490,-0.779410,-0.878846,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [40]:
# x_processed_df.to_csv("processed_weather_data.csv", index=False)

In [41]:
x = df[["Humidity"]]
Y = df["Dryness_Label_Wildfire"]
# Split the data

In [42]:
def define_dryness_for_wildfire(row):
    if row["Precipitation (%)"] < 10 and row["Humidity"] < 50 and row["Temperature"] > 25:
        return "Dry"
    else:
        return "Not Dry"

df["Dryness_Label_Wildfire"] = df.apply(define_dryness_for_wildfire, axis=1)
print("\nValue counts for Dryness_Label_Wildfire:")
print(df["Dryness_Label_Wildfire"].value_counts())

# Target variable for classification (1 for Dry, 0 for Not Dry)
y = (df["Dryness_Label_Wildfire"] == "Dry").astype(int)


Value counts for Dryness_Label_Wildfire:
Dryness_Label_Wildfire
Not Dry    12618
Dry          582
Name: count, dtype: int64


In [43]:
x.shape

(13200, 1)

In [ ]:
x_train, x_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [45]:
#Find the best fit line in Simple linear regression
lr = LinearRegression()
lr.fit(x_train, Y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [46]:
#Predict data
Y_pred = lr.predict(x_test)
Y_pred

array([ 0.19419095, -0.07676115,  0.05690855, ...,  0.11109897,
        0.17612748,  0.11109897], shape=(2640,))

In [47]:
#Find the line slope
lr.coef_

array([-0.00361269])

In [48]:
#FInd the line intercept
lr.intercept_

np.float64(0.29173370767554957)

In [49]:
# plt.scatter(x_train, Y_train)
# plt.show()

In [50]:
# #Plot the best fit line for training data
# plt.scatter(x_train, Y_train)
# plt.plot(x_train, 0.055*x_train + 7.007, 'g')
# plt.title('Best fit line for training data')
# plt.show()

In [51]:
# #Plot the best fit line for test data
# plt.scatter(x_test, Y_test)
# plt.plot(x_test, 0.055*x_test + 7.007, 'g')
# plt.title('Best fit line for test data')
# plt.show()

In [52]:
#Dataframe for predicted and actual values
results_diff = pd.DataFrame({'Predicted Value': Y_pred, 'Actual Value': Y_test})
results_diff

,Predicted Value,Actual Value
1092,0.194191,0
10991,-0.076761,0
6597,0.056909,0
4164,-0.073148,0
445,0.009944,0
...,...,...
4356,0.042458,0
4514,-0.055085,0
9376,0.111099,0
1797,0.176127,0


In [53]:
# #Plot the histogram for the error/cost function
# fig = plt.figure()
# sns.histplot(Y_test - Y_pred)
# plt.xlabel('Y_test - Y_test_pred', fontsize = 15)
# plt.show()

In [54]:
#Compute the metrics
mae = mean_absolute_error(Y_test, Y_pred)
mse = mean_squared_error(Y_test, Y_pred)
r2 = r2_score(Y_test, Y_pred)

print('Mean Absolute Error: ', mae)
print('Mean Squared Error: ', mse)
print('R2 coefficient: ', r2)

Mean Absolute Error:  0.09836115566949943
Mean Squared Error:  0.03885118765674918
R2 coefficient:  0.13207973007436558


### Conclusion:

Out of 60/40 and 90/10 split, 90/10 split is slightly considered a best model, but because the both results are with low percentage the model isn't efficient to predict the Dryness weather conditions.

Another Strategy to try to improve that model is play around with the columns and find a best way to follow the direction to make the model works for what is was made for. 

### 60/40 Split
* Mean Absolute Error:  0.09823481987365398
* Mean Squared Error:  0.037938878941927455
* R2 coefficient:  0.12905458362973743


### 90/10 Split Results

* Mean Absolute Error:  0.09870580611047627
* Mean Squared Error:  0.03879481718476343
* R2 coefficient:  0.13333902427391386

#### Assignment: Second part


Visualizing features

In [55]:
df["Season"].unique()

array(['Winter', 'Spring', 'Summer', 'Autumn'], dtype=object)

In [56]:
d_1 = pd.DataFrame({'Feature': x_processed_df.columns})
d_1

,Feature
0,num__Temperature
1,num__Humidity
2,num__Wind Speed
3,num__Precipitation (%)
4,num__Atmospheric Pressure
5,num__UV Index
6,num__Visibility (km)
7,cat__Cloud Cover_cloudy
8,cat__Cloud Cover_overcast
9,cat__Cloud Cover_partly cloudy


#### Dropping features

- 1 - __TEMPERATURE__

In [57]:
x1 = x_processed_df.drop(["num__Temperature"], axis=1)


In [58]:
x1.shape

(13200, 14)

In [59]:
x1# Split the data again with the reduced feature set
x1_train, x1_test, Y1_train, Y1_test = train_test_split(x1, y, test_size=0.2, random_state=19)

# Multiple Linear Regression with reduced features
lr1 = LinearRegression()  # Model Building
lr1.fit(x1_train, Y1_train)  # Model Training


# Display the intercept of the model
print(f"Intercept / bias: {lr1.intercept_:.4f}")

# Predict data
Y1_pred = lr1.predict(x1_test)

# Compute the metrics
mae1 = mean_absolute_error(Y1_test, Y1_pred)
mse1 = mean_squared_error(Y1_test, Y1_pred)
r2_1 = r2_score(Y1_test, Y1_pred)
print('Mean Absolute Error: ', mae1)
print('Mean Squared Error: ', mse1)
print('R2 coefficient: ', r2_1)



Intercept / bias: 0.1092
Mean Absolute Error:  0.09587426866344077
Mean Squared Error:  0.036351388597698986
R2 coefficient:  0.18792425903083854


In [60]:
# Dataframe for predicted and actual values
results_diff1 = pd.DataFrame({'Predicted Value': Y1_pred, 'Actual Value': Y1_test})
results_diff1

,Predicted Value,Actual Value
1092,0.279747,0
10991,-0.011313,0
6597,0.024467,0
4164,-0.078126,0
445,-0.002357,0
...,...,...
4356,0.023193,0
4514,-0.056560,0
9376,0.089500,0
1797,0.122532,0


In [61]:
# # Plot the histogram for the error/cost function
# fig1 = plt.figure()
# sns.histplot(Y1_test - Y1_pred)
# plt.xlabel('Y_test - Y_test_pred', fontsize=15)
# plt.show()

- 2 - __HUMIDITY__

In [62]:
x2 = x_processed_df.drop(["num__Humidity"], axis=1)


In [63]:
x2.shape

(13200, 14)

In [64]:
# Split the data again with the reduced feature set
x2
x2_train, x2_test, Y2_train, Y2_test = train_test_split(x2, y, test_size=0.2, random_state=19)

# Multiple Linear Regression with reduced features
lr2 = LinearRegression()  # Model Building
lr2.fit(x2_train, Y2_train)  # Model Training

# Display the intercept of the model
print(f"Intercept / bias: {lr2.intercept_:.4f}")

# Predict data
Y2_pred = lr2.predict(x2_test)

# Compute the metrics
mae2 = mean_absolute_error(Y2_test, Y2_pred)
mse2 = mean_squared_error(Y2_test, Y2_pred)
r2_2 = r2_score(Y2_test, Y2_pred)
print('Mean Absolute Error: ', mae2)
print('Mean Squared Error: ', mse2)
print('R2 coefficient: ', r2_2)


Intercept / bias: 0.1053
Mean Absolute Error:  0.09416979800158262
Mean Squared Error:  0.037750682390907224
R2 coefficient:  0.15666458539006167


In [65]:
# Dataframe for predicted and actual values
results_diff2 = pd.DataFrame({'Predicted Value': Y2_pred, 'Actual Value': Y2_test})
results_diff2

,Predicted Value,Actual Value
1092,0.206562,0
10991,0.055225,0
6597,-0.038249,0
4164,-0.048401,0
445,-0.031279,0
...,...,...
4356,0.020882,0
4514,0.005383,0
9376,0.087899,0
1797,0.074280,0


In [66]:
# # Plot the histogram for the error/cost function
# fig2 = plt.figure()
# sns.histplot(Y2_test - Y2_pred)
# plt.xlabel('Y_test - Y_test_pred', fontsize=15)
# plt.show()

- Third: __HUMIDITY__

* 
* __PRECIPITATION__
* __ATMOSPHERIC PRESSURE__
* __UV INDEX__
* __VISIBILITY (KM)__
* __CLOUD COVER_CLOUDY__
* __CLOUD COVER_OVERCAST__
* __CLOUD COVER_PARTLY CLOUDY__
* __SEASON_SPRING__
* __SEASON_SUMMER__
* __SEASON_WINTER__
* __LOCATION_INLAND__
* __LOCATION_MOUNTAIN__

* 4 - __WIND SPEED__

In [67]:
# Model 1:
print("\n--- Model 1 Results ---")
print(f"Intercept: {lr1.intercept_:.4f}")
print(f"Mean Absolute Error: {mae1:.4f}")
print(f"Mean Squared Error: {mse1:.4f}")
print(f"R2 Score: {r2_1:.4f}")
# Model 2:
print("\n--- Model 2 Results ---")
print(f"Intercept: {lr2.intercept_:.4f}")
print(f"Mean Absolute Error: {mae2:.4f}")
print(f"Mean Squared Error: {mse2:.4f}")
print(f"R2 Score: {r2_2:.4f}")


--- Model 1 Results ---
Intercept: 0.1092
Mean Absolute Error: 0.0959
Mean Squared Error: 0.0364
R2 Score: 0.1879

--- Model 2 Results ---
Intercept: 0.1053
Mean Absolute Error: 0.0942
Mean Squared Error: 0.0378
R2 Score: 0.1567
